In [1]:
# Import pandas
import pandas as pd
import numpy as np

# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from pathlib import Path

import tensorflow as tf

# Import Gradio
import gradio as gr

In [2]:
# Set the model's file path
model_path = Path("milking_the_data.keras")

# Load the model to a new object
model = tf.keras.models.load_model(model_path)
model

C:\Users\jclib\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


<Sequential name=sequential, built=True>

In [3]:
import pickle

# Load the scaler from the file
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [4]:
def predict_milk_production(lact_num, avg_thi3, avg_csi3, avg_csi30, avg_precspring,
                            date_diff, calv_month, calv_day_of_month, alp_month, alp_day_of_month):
    # Prepare the input data for the model
    input_data = np.array([[lact_num, avg_thi3, avg_csi3, avg_csi30, avg_precspring,
                            date_diff, calv_month, calv_day_of_month, alp_month, alp_day_of_month]])
    #scale the data
    input_data_scaled = scaler.transform(input_data)
    # Make prediction
    prediction = model.predict(input_data_scaled)
    
    return prediction[0][0]  # Return the predicted value


In [6]:
# Define the Gradio interface
inputs = [
    gr.Number(label='Lactation Number (number of births)'),
    gr.Slider(30, 80, step = 1,label='Average Temp Humidity Index (3 days)'),
    gr.Slider(700, 1300, step = 10, label='Average Cold Stress Index (3 days)'),
    gr.Slider(700, 1300, step = 10, label='Average Cold Stress Index (30 days)'),
    gr.Slider(0,500, step = 10, label='Average Precipitation Spring (mm/month)'),
    gr.Number(label='Number of Days Between Calving and Arriving in the Alp'),
    gr.Slider(1,12,step=1,label='Calving Month (calv_month)'),
    gr.Slider(1,31, step = 1, label='Calving Day of Month (calv_day_of_month)'),
    gr.Slider(1,12,step=1,label='Alping Month (alp_month)'),
    gr.Slider(1,31, step = 1,label='Alping Day of Month (alp_day_of_month)')
]

outputs = gr.Textbox(label='Predicted Milk Production')

interface = gr.Interface(fn=predict_milk_production, inputs=inputs, outputs=outputs)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


C:\Users\jclib\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
